# Test functions in case of polygon rooms( L-shape)
### 1. Find Opposite wall



In [1]:

from shapely.geometry import Polygon, LineString

def extract_walls(polygon: Polygon):
    coords = list(polygon.exterior.coords)
    return [LineString([coords[i], coords[i+1]]) for i in range(len(coords) - 1)]


### 2. Calculate normal vector and direction vector

In [2]:
import numpy as np

def unit_vector(p1, p2):
    v = np.array(p2) - np.array(p1)
    return v / np.linalg.norm(v)

def direction_vector(line):
    p1, p2 = line.coords
    return unit_vector(p1, p2)

def normal_vector(line):
    d = direction_vector(line)
    return np.array([-d[1], d[0]])  # 90° CCW

def wall_midpoint(line):
    p1, p2 = line.coords
    return (np.array(p1) + np.array(p2)) / 2

def are_adjacent(line1, line2):
    return set(line1.coords).intersection(set(line2.coords))


In [3]:
def find_opposite_and_adjacent(walls, angle_tol=15, parallel_tol=0.95):
    result = {}

    for i, wall in enumerate(walls):
        wall_dir = direction_vector(wall)
        wall_norm = normal_vector(wall)
        wall_mid = wall_midpoint(wall)

        adjacents = []
        opposites = []
        distances = []

        for j, other in enumerate(walls):
            if i == j:
                continue

            other_dir = direction_vector(other)
            other_norm = normal_vector(other)
            other_mid = wall_midpoint(other)

            # Adjacency
            if are_adjacent(wall, other):
                adjacents.append((j, other))
                continue  # skip from opposites

            # Check parallelism
            dir_dot = np.dot(wall_dir, other_dir)
            is_parallel = abs(dir_dot) > parallel_tol

            # Normals facing each other
            normal_dot = np.dot(wall_norm, other_norm)
            facing_each_other = normal_dot < -parallel_tol

            if is_parallel and facing_each_other:
                # Compute distance between midpoints projected along normal
                offset = other_mid - wall_mid
                proj = np.dot(offset, wall_norm)
                opposites.append((j, other, proj))

            # Track all distances
            dist = np.linalg.norm(wall_mid - other_mid)
            distances.append((j, dist))

        # Sort opposite by distance
        opposites.sort(key=lambda x: -x[2])  # largest projection along normal
        closest = sorted(distances, key=lambda x: x[1])[:2]

        result[i] = {
            "adjacent_ids": [a[0] for a in adjacents],
            "opposite_ids": [o[0] for o in opposites],
            "closest_ids": [c[0] for c in closest]
        }

    return result


In [11]:
coords = [
    (0, 0), (0, 400), (200, 400),
    (200, 200), (400, 200), (400, 0)
]
room = Polygon(coords)
walls = extract_walls(room)

wall_relations = find_opposite_and_adjacent(walls)


In [6]:
print("wall relations", wall_relations[0])

wall relations {'adjacent_ids': [1, 5], 'opposite_ids': [4, 2], 'closest_ids': [2, 3]}


In [12]:
print("wall relations", wall_relations[0])

wall relations {'adjacent_ids': [1, 5], 'opposite_ids': [2, 4], 'closest_ids': [2, 3]}


### 3. Calculate which wall

In [19]:
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 1e-6 else v

def get_wall_direction(p1, p2, tolerance=0.7):
    dx, dy = np.array(p2) - np.array(p1)
    direction = normalize((dx, dy))

    # Adjusted to screen-style axis: (0,0) top-left
    directions = {
        'right':    np.array([1, 0]),   # horizontal right →
        'left': np.array([-1, 0]),  # horizontal left ←
        'top':  np.array([0, 1]),   # vertical down ↓
        'bottom':   np.array([0, -1])   # vertical up ↑
    }

    best_match = None
    best_score = -1.0
    for name, axis in directions.items():
        score = np.dot(direction, axis)
        if score > best_score and score > tolerance:
            best_match = name
            best_score = score

    return best_match or 'diagonal'


In [20]:
coords = [
    (0, 0), (0, 400), (200, 400),
    (200, 200), (400, 200), (400, 0)
]

# Create wall segments (wrap around)
walls = [(coords[i], coords[(i + 1) % len(coords)]) for i in range(len(coords))]

# Classify each wall
for idx, (p1, p2) in enumerate(walls):
    direction = get_wall_direction(p1, p2)
    print(f"Wall {idx}: {p1} → {p2} → Direction: {direction}")


Wall 0: (0, 0) → (0, 400) → Direction: top
Wall 1: (0, 400) → (200, 400) → Direction: right
Wall 2: (200, 400) → (200, 200) → Direction: bottom
Wall 3: (200, 200) → (400, 200) → Direction: right
Wall 4: (400, 200) → (400, 0) → Direction: bottom
Wall 5: (400, 0) → (0, 0) → Direction: left
